In [1]:
%load_ext autoreload
%autoreload 2

In [42]:
import pandas as pd
import numpy as np
import modules.utils as Utils
from modules.regex_patterns import reviews_paths
from collections import Counter

### 1. Đọc hết tất cả dữ liệu sau khi crawl về vào một dataframe duy nhất

In [3]:
raw_data = pd.DataFrame(columns=['comment', 'rating'])

In [4]:
for path in reviews_paths:
    products = Utils.getFiles(path)
    for product in products:
        df = pd.read_csv(path + product)
        raw_data = pd.concat([raw_data, df], axis=0).reset_index(drop=True)

🚩 **Đổi tên cột `comment` thành `raw_comment`.**

In [5]:
raw_data = raw_data.rename(columns={"comment":"raw_comment"})

In [6]:
raw_data

,raw_comment,rating
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5
...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5
278155,Dởm,5
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5
278157,Hàng đẹp ạ😍,5


🚩 **Xem `dtype` tại các cột.**

In [7]:
raw_data.dtypes

raw_comment    object
rating         object
dtype: object

🚩 **Chuẩn lại kiểu dữ liệu.**

In [8]:
raw_data = raw_data.astype({"raw_comment":"string", "rating":"Int64"})

In [9]:
raw_data.dtypes

raw_comment    string
rating          Int64
dtype: object

In [10]:
raw_data

,raw_comment,rating
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5
...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5
278155,Dởm,5
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5
278157,Hàng đẹp ạ😍,5


### 2. Tạo cột `label` cho `raw_data` theo tiêu chí:
* $\leq 2$: -1 (tiêu cực)
* $\in [3, 4]$: 0 (bình thường)
* $= 5$: 1 (tích cực)

In [11]:
raw_data['label'] = raw_data['rating'].apply(lambda rating: Utils.labelRating(rating))

In [12]:
raw_data

,raw_comment,rating,label
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",5,1
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",5,1
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",5,1
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,5,1
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,5,1
...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,5,1
278155,Dởm,5,1
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,5,1
278157,Hàng đẹp ạ😍,5,1


🚩 **Xóa cột `rating` đi.**

In [13]:
raw_data = raw_data.drop(columns=['rating'])

In [14]:
raw_data

,raw_comment,label
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1
...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1
278155,Dởm,1
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1
278157,Hàng đẹp ạ😍,1


### 3. Extract emoji

In [15]:
raw_data['emoji'] = raw_data['raw_comment'].apply(lambda comment: Utils.extractEmoji(comment))

In [16]:
raw_data

,raw_comment,label,emoji
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,
...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,
278155,Dởm,1,
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,
278157,Hàng đẹp ạ😍,1,😍


### 4. **Tạo cột `processing_comment` bằng cách `.lower()` cột `raw_comment`.**

In [17]:
raw_data['processing_comment'] = raw_data['raw_comment'].apply(lambda comment: comment.lower())

In [18]:
raw_data

,raw_comment,label,emoji,processing_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,"lần đầu mua tại shop nhưng rất ưng ý, vải mặc ..."
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,"giao hàng nhanh, shop xịn xò đặt hàng xong là ..."
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,"size vừa vặn luôn, shop mall mà hàng chất lượn..."
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...
...,...,...,...,...
278154,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm lun mọi ngừ ơi . hdhhechhjhshxhdhhxhxh...
278155,Dởm,1,,dởm
278156,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua 12 món giao đủ. vừa rẻ vừa chất lượng rất ...
278157,Hàng đẹp ạ😍,1,😍,hàng đẹp ạ😍


### 5. Xóa các mẫu chứa URL (khả năng cao là quảng cáo, spam,...)

In [19]:
raw_data['contain_url'] = raw_data['processing_comment'].apply(lambda comment: 1 if Utils.containsURL(comment) else 0)

In [20]:
Counter(raw_data['contain_url'])

Counter({0: 277589, 1: 570})

> **Nhận xét**:
> * Có 570 mẫu tìm ẩn khả năng chứa quảng cáo.<br>
>
> $\Rightarrow $ Xóa các mẫu chứa url.

🚩 **Xóa các mẫu chứa url**

In [21]:
raw_data = raw_data[raw_data['contain_url'] == 0].reset_index(drop=True)

In [22]:
raw_data = raw_data.drop(columns=['contain_url'])

In [23]:
raw_data

,raw_comment,label,emoji,processing_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,"lần đầu mua tại shop nhưng rất ưng ý, vải mặc ..."
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,"giao hàng nhanh, shop xịn xò đặt hàng xong là ..."
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,"size vừa vặn luôn, shop mall mà hàng chất lượn..."
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...
...,...,...,...,...
277584,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm lun mọi ngừ ơi . hdhhechhjhshxhdhhxhxh...
277585,Dởm,1,,dởm
277586,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua 12 món giao đủ. vừa rẻ vừa chất lượng rất ...
277587,Hàng đẹp ạ😍,1,😍,hàng đẹp ạ😍


### 6. Loại bỏ _dấu câu_, _kí tự đặc biệt_, _số_

In [24]:
raw_data['processing_comment'] = raw_data['processing_comment'].apply(lambda comment: Utils.removeNotVietnameseLetters(comment))

In [25]:
raw_data

,raw_comment,label,emoji,processing_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại shop nhưng rất ưng ý vải mặc m...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,giao hàng nhanh shop xịn xò đặt hàng xong là c...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,size vừa vặn luôn shop mall mà hàng chất lượng...
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...
...,...,...,...,...
277584,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm lun mọi ngừ ơi hdhhechhjhshxhdhhxhxhdh...
277585,Dởm,1,,dởm
277586,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món giao đủ vừa rẻ vừa chất lượng rất thíc...
277587,Hàng đẹp ạ😍,1,😍,hàng đẹp ạ


### 7. Chuẩn lại các từ bị duplicate kí tự lên (ví dụ: "anh nhớớớớớớớớớớ emmmmmmmmmmmmmmmmmmmmm quááááááááááááááá" thành "anh nhớ em quá")

In [26]:
raw_data['processing_comment'] = raw_data['processing_comment'].apply(lambda comment: Utils.removeDuplicateLetters(comment))

In [27]:
raw_data

,raw_comment,label,emoji,processing_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại shop nhưng rất ưng ý vải mặc m...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,giao hàng nhanh shop xịn xò đặt hàng xong là c...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,size vừa vặn luôn shop mal mà hàng chất lượng ...
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...
...,...,...,...,...
277584,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm lun mọi ngừ ơi hdhechjhshxhdhxhxhdh hs...
277585,Dởm,1,,dởm
277586,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món giao đủ vừa rẻ vừa chất lượng rất thíc...
277587,Hàng đẹp ạ😍,1,😍,hàng đẹp ạ


### 8. Chuẩn lại một vài từ viết tắt cơ bản

In [28]:
vietnamese_abbreviate = Utils.loadVietnameseAbbreviate()

In [29]:
raw_data['processing_comment'] = raw_data['processing_comment'].apply(lambda comment: Utils.standardVietnameseAbbreviate(vietnamese_abbreviate, comment))

In [30]:
raw_data

,raw_comment,label,emoji,processing_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại shop nhưng rất ưng ý vải mặc m...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,giao hàng nhanh shop xịn xò đặt hàng xong là c...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,size vừa vặn luôn shop mal mà hàng chất lượng ...
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...
...,...,...,...,...
277584,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm luôn mọi người ơi hdhechjhshxhdhxhxhdh...
277585,Dởm,1,,dởm
277586,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món giao đủ vừa rẻ vừa chất lượng rất thíc...
277587,Hàng đẹp ạ😍,1,😍,hàng đẹp ạ


### 9. Tạo cột `boost_words` chứa các từ chuyên ngành

In [31]:
boost_words = Utils.loadBoostWords()

In [32]:
raw_data['boost_words'] = raw_data['processing_comment'].apply(lambda comment: Utils.extractBoostWords(boost_words, comment))

In [33]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại shop nhưng rất ưng ý vải mặc m...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,giao hàng nhanh shop xịn xò đặt hàng xong là c...,giao_hàng_nhanh chất_vải cẩn_thận ok xịn_xò
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,size vừa vặn luôn shop mal mà hàng chất lượng ...,giá_rẻ chất_lượng vừa_vặn
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng đóng gói kĩ shop nhiệt tìn...,ủng_hộ đóng_gói chất_lượng sản_phẩm
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...
...,...,...,...,...,...
277584,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm luôn mọi người ơi hdhechjhshxhdhxhxhdh...,mọi_người đẹp_lắm_luôn
277585,Dởm,1,,dởm,
277586,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món giao đủ vừa rẻ vừa chất lượng rất thíc...,ủng_hộ chất_lượng giao_đủ
277587,Hàng đẹp ạ😍,1,😍,hàng đẹp ạ,hàng_đẹp


### 10. Loại bỏ các từ rác ra khỏi comment, (ví dụ: "asfsfskjfhfjkfhf", "aaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaaa")

In [34]:
vietnamese_syllables = Utils.loadVietnameseSyllables()

In [35]:
raw_data['processing_comment'] = raw_data['processing_comment'].apply(lambda comment: Utils.removeGibbish(vietnamese_syllables, comment))

In [36]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại nhưng rất ưng ý vải mặc mát vì...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,giao hàng nhanh xịn xò đặt hàng xong là chủ độ...,giao_hàng_nhanh chất_vải cẩn_thận ok xịn_xò
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,vừa vặn luôn mà hàng chất lượng giá rẻ vầy tội...,giá_rẻ chất_lượng vừa_vặn
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng đóng gói kĩ nhiệt tình sẽ ...,ủng_hộ đóng_gói chất_lượng sản_phẩm
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...
...,...,...,...,...,...
277584,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm luôn mọi người ơi học,mọi_người đẹp_lắm_luôn
277585,Dởm,1,,dởm,
277586,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món giao đủ vừa rẻ vừa chất lượng rất thíc...,ủng_hộ chất_lượng giao_đủ
277587,Hàng đẹp ạ😍,1,😍,hàng đẹp ạ,hàng_đẹp


### 11. Xóa các mẫu bị duplicate

In [37]:
raw_data['tmp_comment'] = raw_data.apply(lambda row: Utils.combineCommentAndEmoji(row['processing_comment'], row['emoji']), axis=1)

In [38]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words,tmp_comment
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại nhưng rất ưng ý vải mặc mát vì...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...,lần đầu mua tại nhưng rất ưng ý vải mặc mát vì...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,giao hàng nhanh xịn xò đặt hàng xong là chủ độ...,giao_hàng_nhanh chất_vải cẩn_thận ok xịn_xò,giao hàng nhanh xịn xò đặt hàng xong là chủ độ...
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,vừa vặn luôn mà hàng chất lượng giá rẻ vầy tội...,giá_rẻ chất_lượng vừa_vặn,vừa vặn luôn mà hàng chất lượng giá rẻ vầy tội...
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng đóng gói kĩ nhiệt tình sẽ ...,ủng_hộ đóng_gói chất_lượng sản_phẩm,sản phẩm chất lượng đóng gói kĩ nhiệt tình sẽ ...
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...
...,...,...,...,...,...,...
277584,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm luôn mọi người ơi học,mọi_người đẹp_lắm_luôn,đẹp lắm luôn mọi người ơi học
277585,Dởm,1,,dởm,,dởm
277586,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món giao đủ vừa rẻ vừa chất lượng rất thíc...,ủng_hộ chất_lượng giao_đủ,mua món giao đủ vừa rẻ vừa chất lượng rất thíc...
277587,Hàng đẹp ạ😍,1,😍,hàng đẹp ạ,hàng_đẹp,hàng đẹp ạ 😍


In [39]:
raw_data = raw_data.drop_duplicates(subset=['tmp_comment']).drop(columns=['tmp_comment']).reset_index(drop=True)

In [40]:
raw_data

,raw_comment,label,emoji,processing_comment,boost_words
0,"Lần đầu mua tại shop nhưng rất ưng ý, vải mặc ...",1,,lần đầu mua tại nhưng rất ưng ý vải mặc mát vì...,giao_hàng giá_tiền ủng_hộ cẩn_thận đóng_gói lâ...
1,"Giao hàng nhanh, shop xịn xò đặt hàng xong là ...",1,,giao hàng nhanh xịn xò đặt hàng xong là chủ độ...,giao_hàng_nhanh chất_vải cẩn_thận ok xịn_xò
2,"Size vừa vặn luôn, shop mall mà hàng chất lượn...",1,,vừa vặn luôn mà hàng chất lượng giá rẻ vầy tội...,giá_rẻ chất_lượng vừa_vặn
3,Sản phẩm chất lượng.đóng gói kĩ.shop nhiệt tìn...,1,,sản phẩm chất lượng đóng gói kĩ nhiệt tình sẽ ...,ủng_hộ đóng_gói chất_lượng sản_phẩm
4,Chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,1,,chất lượng sản phẩm tuyệt vời giao hàng nhanh ...,giao_hàng_nhanh ủng_hộ tuyệt_vời chất_lượng sả...
...,...,...,...,...,...
220023,Nơ xinh đúng chất công chúa đẹpppppp,1,,nơ xinh đúng chất công chúa đẹp,
220024,"Rất dễ thương, đẹp",1,,rất dễ thương đẹp,dễ_thương
220025,Đẹp lắm lun mọi ngừ ơi . Hdhhechhjhshxhdhhxhxh...,1,,đẹp lắm luôn mọi người ơi học,mọi_người đẹp_lắm_luôn
220026,Mua 12 món giao đủ. Vừa rẻ vừa chất lượng rất ...,1,,mua món giao đủ vừa rẻ vừa chất lượng rất thíc...,ủng_hộ chất_lượng giao_đủ


### 12. Xóa các `processing_comment` bị `nan` hoặc `""`

In [43]:
raw_data = raw_data.replace("", np.nan)
raw_data = raw_data.dropna(subset=['processing_comment']).reset_index(drop=True)

### 13. Ghi ra file

In [44]:
raw_data.to_csv("./data/raw_data.csv", index=False)